In [1]:
## Imports
from __future__ import print_function

import torch
from torch import optim
from torch.utils.data import DataLoader
import torch.nn as nn

from unet.unet_model import UNet

from utils.dataset import CrossValDataset
from utils.model_eval import dice_coefficient

from torch.utils.tensorboard import SummaryWriter

In [2]:
## Device
if torch.cuda.is_available():
    device = 'cuda'
    print('Using GPU')
else:
    device = 'cpu'
    print('Using CPU')

Using GPU


In [3]:
## Dataset
groupPath = "D:\\autoseg\\train"
batch_size = 8

## Training
n_channels = 1
n_classes = 13

learning_rate = 1e-3
weight_decay  = 1e-8
momentum      = 0.9

num_epochs    = 41

In [4]:
## Train
Npartition = 5
for partition in range(5):
    trainDataset = CrossValDataset(groupPath,partition, testFlag = False)
    testDataset  = CrossValDataset(groupPath,partition, testFlag = True)
    trainLoader  = DataLoader(trainDataset,batch_size=batch_size)
    testLoader   = DataLoader(testDataset,batch_size=batch_size)
    
    # Define Net
    net = UNet(n_channels=n_channels,n_classes=n_classes)
    net.to(device=device)
    
    # Optimizer
    optimizer = optim.RMSprop(net.parameters(),lr=learning_rate,weight_decay=weight_decay,momentum=momentum)
    critereon = nn.CrossEntropyLoss()
    
    #Tensorboard
    writer = SummaryWriter('runs/full_model_partition_'+str(partition))
    
    for epoch in range(num_epochs):
        net.train()
        epoch_train_loss = 0
        for trainIndex,batch in enumerate(trainLoader,0):
            imgs    = batch['img']
            segs_gt = batch['seg']
            
            imgs    = imgs.to(   device=device,dtype=torch.float32)
            segs_gt = segs_gt.to(device=device,dtype=torch.long)
            
            segs_pr = net(imgs)
            
            loss = critereon(segs_pr,segs_gt)
            epoch_train_loss += loss.item()/len(trainLoader)
            
            optimizer.zero_grad()
            loss.backward()
            nn.utils.clip_grad_value_(net.parameters(),0.1)
            optimizer.step()
            
            print("Train: " + str(loss.item()))
            
        writer.add_scalar('Training Loss',
                        epoch_train_loss,
                        epoch)
        
        net.eval()
        epoch_test_loss = 0
        for testIndex,batch in enumerate(testLoader,0):
            imgs    = batch['img']
            segs_gt = batch['seg']
            
            imgs    = imgs.to(   device=device,dtype=torch.float32)
            segs_gt = segs_gt.to(device=device,dtype=torch.long)
            
            segs_pr = net(imgs)
            
            loss = critereon(segs_pr,segs_gt)
            epoch_test_loss += loss.item()/len(testLoader)
            
            print("Test: " + str(loss.item()))
            
        writer.add_scalar('Validation Loss',
                        epoch_test_loss,
                        epoch)
        
        savefile = "D:\\autoseg\\nets\\unet-save" + str(partition) + "_" + str(epoch) + ".pth"
        torch.save(net.state_dict(),savefile)

Indexing data directory: D:\autoseg\train...
5 groups found.
Partition selected: 0
Serving training data.
Indexing data directory: D:\autoseg\train...
5 groups found.
Partition selected: 0
Serving test data.
Train: 2.677539825439453
Train: 2.3721394538879395
Train: 1.410618782043457
Train: 1.064863920211792
Train: 0.7592030763626099
Train: 0.7663349509239197
Train: 0.6422021389007568
Train: 0.582194447517395
Train: 0.5763294100761414
Train: 0.5528571009635925
Train: 0.4977993071079254
Train: 0.4794374108314514
Train: 0.4851253628730774
Train: 0.4421243667602539
Train: 0.4295881390571594
Train: 0.4207993447780609
Train: 0.3946882486343384
Train: 0.3665114641189575
Train: 0.3648921549320221
Train: 0.3324025571346283
Train: 0.4504736363887787
Train: 0.37847453355789185
Train: 0.3533192574977875
Train: 0.34990161657333374
Train: 0.34795671701431274
Train: 0.3174184560775757
Train: 0.297320693731308
Test: 61.60360336303711
Test: 60.094947814941406
Test: 58.75881576538086
Test: 61.5832443237

Test: 0.41676831245422363
Test: 0.5071994662284851
Train: 0.08793207257986069
Train: 0.09122362732887268
Train: 0.07766204327344894
Train: 0.06915255635976791
Train: 0.08223766833543777
Train: 0.06431663781404495
Train: 0.06498625129461288
Train: 0.0668393075466156
Train: 0.06546859443187714
Train: 0.06166078895330429
Train: 0.058468133211135864
Train: 0.0564674511551857
Train: 0.05647243559360504
Train: 0.05554472282528877
Train: 0.06027873232960701
Train: 0.05950692668557167
Train: 0.056541021913290024
Train: 0.045325081795454025
Train: 0.050961073487997055
Train: 0.04664232209324837
Train: 0.07256899774074554
Train: 0.06730043888092041
Train: 0.0651545375585556
Train: 0.06286843121051788
Train: 0.06534130871295929
Train: 0.06315772980451584
Train: 0.05670064687728882
Test: 0.4254955053329468
Test: 0.7770636081695557
Test: 0.7618853449821472
Test: 0.09623938053846359
Test: 0.7785186171531677
Test: 0.4224376678466797
Test: 0.5461632609367371
Train: 0.07238179445266724
Train: 0.0678628

Test: 0.4848437011241913
Test: 0.6243994832038879
Train: 0.04199958220124245
Train: 0.043889306485652924
Train: 0.04879958555102348
Train: 0.04331742227077484
Train: 0.03783131763339043
Train: 0.0382150262594223
Train: 0.03787843883037567
Train: 0.047231633216142654
Train: 0.04283294454216957
Train: 0.03660697862505913
Train: 0.03340014070272446
Train: 0.03593118116259575
Train: 0.03253748640418053
Train: 0.030271630734205246
Train: 0.03535682335495949
Train: 0.038276586681604385
Train: 0.0410647876560688
Train: 0.0264285858720541
Train: 0.031380731612443924
Train: 0.031170977279543877
Train: 0.04226776957511902
Train: 0.03838836029171944
Train: 0.037576932460069656
Train: 0.04279337078332901
Train: 0.0494401752948761
Train: 0.03702642768621445
Train: 0.033001091331243515
Test: 0.5248485207557678
Test: 0.9479613304138184
Test: 0.9059982895851135
Test: 0.18105432391166687
Test: 0.9341635704040527
Test: 0.5443048477172852
Test: 0.6809375882148743
Train: 0.038522377610206604
Train: 0.0398

Test: 0.7070251107215881
Test: 0.9155502915382385
Train: 0.044602666050195694
Train: 0.05801526457071304
Train: 0.04720897972583771
Train: 0.04975098371505737
Train: 0.06461375206708908
Train: 0.043375443667173386
Train: 0.053527895361185074
Train: 0.06018328294157982
Train: 0.05187668278813362
Train: 0.0423128567636013
Train: 0.04295109212398529
Train: 0.04066029191017151
Train: 0.04020126536488533
Train: 0.03602848947048187
Train: 0.039207618683576584
Train: 0.04119569808244705
Train: 0.04166959971189499
Train: 0.028669122606515884
Train: 0.033983323723077774
Train: 0.031135674566030502
Train: 0.045153889805078506
Train: 0.03786477446556091
Train: 0.03576505556702614
Train: 0.03972101956605911
Train: 0.05199336260557175
Train: 0.039211589843034744
Train: 0.0326324962079525
Test: 0.7937745451927185
Test: 1.2963447570800781
Test: 1.3038846254348755
Test: 0.2968718409538269
Test: 1.2992695569992065
Test: 0.8128331899642944
Test: 0.9629910588264465
Train: 0.04293922707438469
Train: 0.039

Test: 1.5867326259613037
Test: 0.8616965413093567
Test: 1.1335147619247437
Train: 0.03290717676281929
Train: 0.032596033066511154
Train: 0.03458284214138985
Train: 0.0350307896733284
Train: 0.02579936571419239
Train: 0.03421275317668915
Train: 0.03468506410717964
Train: 0.033157773315906525
Train: 0.03341620787978172
Train: 0.02603212743997574
Train: 0.027178026735782623
Train: 0.028108909726142883
Train: 0.027058226987719536
Train: 0.02402566745877266
Train: 0.03120511770248413
Train: 0.026745811104774475
Train: 0.030777649953961372
Train: 0.021809736266732216
Train: 0.02381579391658306
Train: 0.0217011496424675
Train: 0.032296545803546906
Train: 0.029104238376021385
Train: 0.030060352757573128
Train: 0.03438322991132736
Train: 0.03982499986886978
Train: 0.029790667816996574
Train: 0.024883247911930084
Test: 0.7558841109275818
Test: 1.3944560289382935
Test: 1.3443694114685059
Test: 0.16549350321292877
Test: 1.3927924633026123
Test: 0.738041341304779
Test: 0.9766924977302551
Train: 0.0

...

Train: 0.3520506024360657
Train: 0.31949731707572937
Train: 0.038685623556375504
Train: 0.27892446517944336
Train: 0.13912615180015564
Train: 0.13045546412467957
Train: 0.061984915286302567
Train: 0.11282602697610855
Train: 0.06752244383096695
Train: 0.0757627785205841
Train: 0.09033314883708954
Train: 0.05994024872779846
Train: 0.06484776735305786
Train: 0.06315884739160538
Train: 0.06834970414638519
Train: 0.048624537885189056
Train: 0.04476962983608246
Train: 0.04096222296357155
Train: 0.03953416272997856
Train: 0.062266357243061066
Train: 0.044266801327466965
Train: 0.03952190652489662
Train: 0.04143761098384857
Train: 0.06840337067842484
Train: 0.0411527045071125
Train: 0.03307940065860748
Test: 0.12856332957744598
Test: 0.1491839438676834
Test: 0.14632314443588257
Test: 0.1583711951971054
Test: 0.16219286620616913
Test: 0.13002535700798035
Test: 0.14780466258525848
Train: 0.20973610877990723
Train: 0.39968645572662354
Train: 0.37121352553367615
Train: 0.034323304891586304
Train: 

Test: 0.25213178992271423
Test: 0.24361293017864227
Train: 0.2724580466747284
Train: 0.3613569736480713
Train: 0.3307754695415497
Train: 0.11480475217103958
Train: 0.3091985583305359
Train: 0.215785950422287
Train: 0.1994575560092926
Train: 0.09775756299495697
Train: 0.12435264140367508
Train: 0.11342401057481766
Train: 0.11187804490327835
Train: 0.1136096939444542
Train: 0.10212726145982742
Train: 0.11166124045848846
Train: 0.11508572101593018
Train: 0.11074037849903107
Train: 0.10531643033027649
Train: 0.10148950666189194
Train: 0.09956670552492142
Train: 0.08986940234899521
Train: 0.09706872701644897
Train: 0.09122033417224884
Train: 0.09051597118377686
Train: 0.10171731561422348
Train: 0.08328835666179657
Train: 0.07972058653831482
Train: 0.08237893879413605
Test: 0.15310505032539368
Test: 0.14518675208091736
Test: 0.1310502588748932
Test: 0.1478807032108307
Test: 0.1480877548456192
Test: 0.15179942548274994
Test: 0.15155334770679474
Train: 0.23956137895584106
Train: 0.341182231903

Train: 0.18225809931755066
Train: 0.28252649307250977
Train: 0.2625524699687958
Train: 0.054840438067913055
Train: 0.2593790888786316
Train: 0.1427965760231018
Train: 0.1378769725561142
Train: 0.09071871638298035
Train: 0.14379477500915527
Train: 0.12106841057538986
Train: 0.1110541969537735
Train: 0.10602284967899323
Train: 0.09194733947515488
Train: 0.10034574568271637
Train: 0.09628141671419144
Train: 0.0816916823387146
Train: 0.07261959463357925
Train: 0.06672424077987671
Train: 0.06385666131973267
Train: 0.06451354175806046
Train: 0.0780322402715683
Train: 0.06229950487613678
Train: 0.05913887172937393
Train: 0.06271661818027496
Train: 0.05082091689109802
Train: 0.05550823360681534
Train: 0.05179719626903534
Test: 0.09108170866966248
Test: 0.08598178625106812
Test: 0.08648084849119186
Test: 0.08769651502370834
Test: 0.09653094410896301
Test: 0.08638136833906174
Test: 0.08223044127225876
Train: 0.21433669328689575
Train: 0.3186210095882416
Train: 0.2981651723384857
Train: 0.0635952

Train: 0.15814191102981567
Train: 0.24871918559074402
Train: 0.23381201922893524
Train: 0.04635920375585556
Train: 0.23158328235149384
Train: 0.1310146301984787
Train: 0.1341804414987564
Train: 0.04939935728907585
Train: 0.0516662634909153
Train: 0.046081218868494034
Train: 0.04803954064846039
Train: 0.04563157260417938
Train: 0.046485934406518936
Train: 0.05213190242648125
Train: 0.054771095514297485
Train: 0.048853129148483276
Train: 0.045148976147174835
Train: 0.04644150286912918
Train: 0.04412123188376427
Train: 0.04452105239033699
Train: 0.04595819488167763
Train: 0.04084961116313934
Train: 0.037161197513341904
Train: 0.041756607592105865
Train: 0.03413201868534088
Train: 0.03721191734075546
Train: 0.034878846257925034
Test: 0.12270640581846237
Test: 0.11091210693120956
Test: 0.10347934067249298
Test: 0.11503944545984268
Test: 0.12326021492481232
Test: 0.11956334114074707
Test: 0.11160460114479065
Train: 0.15822775661945343
Train: 0.24379201233386993
Train: 0.23091936111450195
Tra

Train: 0.15431801974773407
Train: 0.23904390633106232
Train: 0.22331638634204865
Train: 0.04640096426010132
Train: 0.2236185371875763
Train: 0.12766805291175842
Train: 0.1293763965368271
Train: 0.043073855340480804
Train: 0.047927580773830414
Train: 0.045901115983724594
Train: 0.04714119806885719
Train: 0.04148280993103981
Train: 0.045645032078027725
Train: 0.0524783693253994
Train: 0.0472484827041626
Train: 0.0442003533244133
Train: 0.04471513628959656
Train: 0.04356249049305916
Train: 0.04521150514483452
Train: 0.038540519773960114
Train: 0.04234873503446579
Train: 0.037547528743743896
Train: 0.03906676173210144
Train: 0.04393492266535759
Train: 0.032443877309560776
Train: 0.034295808523893356
Train: 0.0347844660282135
Test: 0.13091979920864105
Test: 0.11532147973775864
Test: 0.11201198399066925
Test: 0.11946318298578262
Test: 0.12882672250270844
Test: 0.1279306262731552
Test: 0.11701608449220657
Train: 0.1566290259361267
Train: 0.23534198105335236
Train: 0.2203795611858368
Train: 0.